## Split data into chunks

Each chunk has no missing moisture measurement for any hour

#### Input:
- ts_data_pp01: preprocessed data from v01.preprocess01.ipynb

#### output:
- ts_data_pp01_chunked_moist

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt
import gc
import dask.dataframe as dd

%matplotlib inline

In [2]:
df_all = pd.read_pickle('../data/ts_data_pp01.pkl')

,LocId,Datetime,Moisture,Temperature,Light,Battery,Latitude,Longitude,ZeroMoist
7490,6,2018-10-12 14:00:00,7.580000,19.719999,5.763333,97.0,47.594,19.362,False
7491,6,2018-10-12 15:00:00,7.635000,NaN,1.932500,97.0,47.594,19.362,False
7492,6,2018-10-12 16:00:00,7.445000,NaN,0.100000,97.0,47.594,19.362,False
7493,6,2018-10-12 17:00:00,7.447500,NaN,0.100000,97.0,47.594,19.362,False
7494,6,2018-10-12 18:00:00,7.385000,NaN,0.100000,97.0,47.594,19.362,False
...,...,...,...,...,...,...,...,...,...
3984239,2075,2019-10-27 11:00:00,33.165001,27.302500,NaN,91.0,53.772,-1.911,False
3984240,2075,2019-10-27 12:00:00,33.227501,16.365000,NaN,91.0,53.772,-1.911,False
3984241,2075,2019-10-27 13:00:00,32.910000,24.635000,NaN,91.0,53.772,-1.911,False
3984242,2075,2019-10-27 14:00:00,32.767498,20.270000,NaN,91.0,53.772,-1.911,False


In [4]:
df_all.shape

(2348080, 9)

In [4]:
df_all = df_all.sort_values(['LocId', 'Datetime'])
df_all = df_all.reset_index(drop=True)

In [5]:
df_all['DtDiff'] = df_all.groupby('LocId').Datetime.diff()
df_all['DtDiff'] = df_all['DtDiff'].apply(lambda x: x.total_seconds() / 3600)

In [6]:
thr_hours = 1

In [7]:
df_all['ChunkBegin'] = (df_all.DtDiff > thr_hours) | (df_all.DtDiff.isnull())
df_all['ChunkId'] = df_all.ChunkBegin.cumsum()

In [8]:
df_all['ChunkId'] = df_all['ChunkId'] - 1

In [9]:
df_all.ChunkId.nunique()

5239

In [10]:
del df_all['DtDiff']

In [11]:
df_all.to_pickle('../data/ts_data_pp01_chunked.pkl')

In [12]:
df_all.columns

Index(['LocId', 'Datetime', 'Moisture', 'Temperature', 'Light', 'Battery',
       'Latitude', 'Longitude', 'ZeroMoist', 'ChunkBegin', 'ChunkId'],
      dtype='object')

In [13]:
del df_all['Temperature']
del df_all['Light']
del df_all['Battery']

In [14]:
df_all.to_pickle('../data/ts_data_pp01_chunked_moist.pkl')